In [3]:
import gpxpy
import gpxpy.gpx
import folium
import folium.plugins as plugins
import time
import os
import json

from folium.plugins import HeatMapWithTime

import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
from IPython.display import display



In [245]:
def read_gpx(file):
    dir="C:/Users/mark_/Documents/GitHub/GPX"
    file=file

    path = os.path.join(dir, file)
    with open(path, 'r') as gpx_file:
        gpx_returned = gpxpy.parse(gpx_file)
    return gpx_returned

In [304]:
def extract_route(gpx_file, color):
    route = []

    for track in gpx_file.tracks:
        for segment in track.segments:
            for point in segment.points:
                route.append({
                    'datetime': point.time,
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation
                })

    df_route = pd.DataFrame(route)
    df_route.set_index('datetime', inplace=True)    
    df_route['timestr'] = df_route.index.strftime('%H:%M:%S')
    df_route['color'] = color
    return df_route
            

In [302]:
gpx_dave = read_gpx("dave_ride.gpx")
gpx_rob = read_gpx("rob_ride.gpx")
gpx_mark = read_gpx("my_ride.gpx")
gpx_simon = read_gpx("simon_ride.gpx")


In [305]:
df_mark = extract_route(gpx_mark, 'blue')
df_rob = extract_route(gpx_rob, 'green')
df_dave = extract_route(gpx_dave, 'red')
df_simon = extract_route(gpx_simon, 'yellow')

In [290]:
# df_mine = pd.DataFrame(my_route)
# df_his = pd.DataFrame(his_route)
# df_rob = pd.DataFrame(rob_route)

# df_mine.set_index('datetime', inplace=True)
# df_his.set_index('datetime', inplace=True)
# df_rob.set_index('datetime', inplace=True)

In [291]:
# df_his.index
# df_his.index = df_his.index + pd.DateOffset(seconds=-2)
# df_mine.index = df_mine.index + pd.DateOffset(seconds=1)


In [307]:
start_time = '10:25:00'
end_time = '10:36:22'

route_map = folium.Map(
    location=[50.9502793, -0.7530442],
    zoom_start=18,
    tiles='OpenStreetMap',
    width=1024,
    height=600
)

def add_to_map(df):
    for _, row in df.between_time(start_time,end_time).iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=4,
            color=row['color'],
            popup=[row['timestr']],
        ).add_to(route_map)

add_to_map(df_mark)
add_to_map(df_dave)
add_to_map(df_simon)
add_to_map(df_rob)

display(route_map)


In [315]:
route_map = folium.Map(
    location=[50.9502793, -0.7530442],
    zoom_start=15,
    tiles='OpenStreetMap',
    width=1024,
    height=600
)

def add_polyline(df, color):
    coordinates = [tuple(x) for x in df.between_time(start_time, end_time)[['latitude', 'longitude']].to_numpy()]
    folium.PolyLine(coordinates, weight=4, color=color).add_to(route_map)

add_polyline(df_mark, 'blue')
add_polyline(df_rob, 'green')
add_polyline(df_dave, 'red')
# coordinates = [tuple(x) for x in df_his.between_time(start_time, end_time)[['latitude', 'longitude']].to_numpy()]
# folium.PolyLine(coordinates, weight=4, color='red').add_to(route_map)

# coordinates = [tuple(x) for x in df_rob.between_time(start_time, end_time)[['latitude', 'longitude']].to_numpy()]
# folium.PolyLine(coordinates, weight=4, color='green').add_to(route_map)

display(route_map)

In [313]:
df_mark.color.unique()

array(['blue'], dtype=object)

In [324]:
start_time = '10:34:59'
end_time = '10:36:22'

route_map = folium.Map(
    location=[50.9502793, -0.7530442],
    zoom_start=18,
    tiles='OpenStreetMap',
    width=1024,
    height=600
)

def divIcon(df):
    for _, row in df.between_time(start_time,end_time).iterrows():
        folium.Marker(location=[row['latitude'], row['longitude']],
            # Set icon to DivIcon to with conditional style formatting to reference the random temp value.
            icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: {row['color']}">{row['timestr']}</div>""")
            ).add_to(route_map)

for df in [df_mark,df_dave, df_rob]:
    divIcon(df)


# for _, row in df_his.between_time(start_time,end_time).iterrows():
#     folium.Marker(location=[row['latitude'], row['longitude']],
#         # Set icon to DivIcon to with conditional style formatting to reference the random temp value.
#         icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: {'red'}">{row['timestr']}</div>""")
#         ).add_to(route_map)

# for _, row in df_rob.between_time(start_time,end_time).iterrows():
#     folium.Marker(location=[row['latitude'], row['longitude']],
#         # Set icon to DivIcon to with conditional style formatting to reference the random temp value.
#         icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: {'green'}">{row['timestr']}</div>""")
#         ).add_to(route_map)

display(route_map)

In [78]:
def create_geojson_features(df, color):

    features = []
    for lat,lan,time in zip(df['latitude'],df['longitude'], df.index): 
        time = str(time)
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[lan,lat]
            },
            'properties': {
                'time': time,
                'style': {'Color' : color},
                'icon': 'Circle',
                'color':color,
                'iconstyle':{
                    'Color': color,
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 7
                }
            }
        }
        features.append(feature)
    return features
features = create_geojson_features(df_his.between_time(start_time, end_time), 'blue')


In [80]:
from folium.plugins import TimestampedGeoJson
m = folium.Map(location=[50.9502793, -0.7530442], zoom_start=18)

TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features}
        , period='PT1H'
        , duration='PT1H'
        , transition_time=10
        , add_last_point=False
        , auto_play=False
        , loop=False
        , max_speed=1
        , loop_button=True
        , date_options='YYYY/MM/DD HH:mm:ss'
        , time_slider_drag_update=True
    ).add_to(m)

display(m) # display the map

In [60]:
lat_long_list = []
temp=[]
for index, instance in df_mine.between_time('10:36:10','10:36:19').iterrows():
    temp.append([instance['latitude'],instance['longitude']])
lat_long_list.append(temp)

In [194]:
# this zips the dataframe variables instead of using iterrows:
# allows you to reference the variable names easier
# need to work out how to provide color - probably build 1 dataframe with a column for 

start_time = '10:36:00'
end_time = '10:36:22'

route_map = folium.Map(
    location=[50.9502793, -0.7530442],
    zoom_start=18,
    tiles='OpenStreetMap',
    width=1024,
    height=600
)

def add_markers(df):
    for lat, lon, timestr in zip(df['latitude'], df['longitude'], df['timestr']):
        folium.CircleMarker(
            location=[lat, lon],
            radius=4,
            popup=timestr,
        ).add_to(route_map)

for df in [df_mine, df_his, df_rob]:
    add_markers(df.between_time(start_time,end_time))

display(route_map)


In [200]:
route_map = folium.Map(
    location=[50.9502793, -0.7530442],
    zoom_start=15,
    tiles='OpenStreetMap',
    width=1024,
    height=600
)

for i in range(n_track):
    folium.CircleMarker(
        location=[df_mine.iloc[i]['latitude'], df_mine.iloc[i]['longitude']],
        radius=3,
        popup=[df_mine.iloc[i]['timestr']],
    ).add_to(route_map)
#     time.sleep(1)
display(route_map)

In [187]:
df_mine.iloc[0]['latitude']

50.944559